In [ ]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.callbacks import TensorBoard, ModelCheckpoint,EarlyStopping,CSVLogger
from keras.layers.advanced_activations import LeakyReLU
from datetime import datetime
#This is the basic model using unresized 200x300 images. 

#Variables to edit-----------------
epochs=90                #Number of epochs for training
filenameprefix = 'CNN200x300ep'+str(epochs) #Start of filename that will be used in saved models

learningrate = 0.0005  #Learning rate for optimizer

espatience = 30          #tensorboard Earlystop function patience value --Use if EarlyStopping is used
#End of variables to edit----------
#filename = filenameprefix + str(datetime.now().strftime('%Y%m%d-%H%M%S'))

#Tensorboard functions
tensorboard = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

checkpointvacc  = ModelCheckpoint('weights/{epoch:02d}.h5'
                             , monitor='val_acc', mode='max', save_best_only=True)
checkpointvloss  = ModelCheckpoint('weights/{epoch:02d}.h5'
                             , monitor='val_loss', mode='min', save_best_only=True)
earlystop   = EarlyStopping(monitor='val_acc', patience=espatience, mode='auto', verbose=1)
csv_logger = CSVLogger(filename+'training.csv', separator=',')


train_data = ImageDataGenerator(rescale=1./255)
test_data = ImageDataGenerator(rescale=1./255)

#End of tensorboard functions
train_generator = train_data.flow_from_directory('../images/train',batch_size=40,class_mode='binary',target_size=(200,300))
test_generator = test_data.flow_from_directory('../images/test',batch_size=32,class_mode='binary',target_size=(200,300))
label_map = (train_generator.class_indices)#Stores classes and their values. Can be displayed on screen if needed

#Model Layers-Layers represent the factors of 200 & 300 for easier calculations
model = Sequential()
model.add(Conv2D(96, (5, 5), strides=5, input_shape=(200, 300, 3)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(128, (5, 5), strides=5))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))      
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
#End of Model layers------------------------------------------------------------
adam = optimizers.adam(lr=learningrate)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
history=model.fit_generator( train_generator,
        steps_per_epoch=60,
        epochs=epochs,
        validation_data=test_generator,
        validation_steps=20, verbose = 1, callbacks=[checkpointvacc,checkpointvloss])#Add tensorboard functions here
        

#model.summary()
filename = filenameprefix + str(datetime.now().strftime('%Y%m%d-%H%M%S')) #Get time and date now to add to filename
model.save('weights/%s.h5' % filename) #Save model

#------------------------------------------------------------------------------
# Plot training & validation accuracy values by history saved-doesn't use tensorboard
plt.figure(1,figsize=(10,6))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure(2,figsize=(10,6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
